In [ ]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [ ]:
import pandas as pd
import numpy as np
import os
import django
from django.db.models import Max, Min, Avg, Q, F
from asgiref.sync import sync_to_async
import tqdm
from collections import defaultdict
from pandarallel import pandarallel
import requests
import json
from matplotlib import pyplot as plt

import ctypes
from ctypes import c_char_p, cdll
GoInt64 = ctypes.c_int64
GoInt = GoInt64
archive_node = "http://localhost:19545"

from etherscan.utils.parsing import ResponseParser as parser
pandarallel.initialize(progress_bar=True)
# os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
# os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
# django.setup()

from debtmonitor.models import *
from datavisualization.models import *
from datastorage.models import *
from debtmonitor.help_function import *

import debtmonitor.views as dm
import datavisualization.views as dv
import datastorage.views as ds
import oracleWeb.views as ow

from debtmonitor.debt_function import *

import pickle

pandarallel.initialize(progress_bar=True)

In [ ]:
def gen_hf_chart(TargetContract, ReservesStatusEnd, StepAhead, MCAmount, ReservesStatusEndIndex, PreviousBlockForTrain):

    reserves_status = get_reserves_status()
    latest_block_num = reserves_status['block_num'].max()
    interaction_df = get_interaction_data(TargetContract)

    until_block_num = latest_block_num
    until_block_num = ReservesStatusEnd
    until_index = ReservesStatusEndIndex

    until_block_n_index = combine_block_n_index({'block_num': until_block_num, 'index': until_index})
    # liquidation_index = until_index

    # Start Getting Data #####################################
    liquidation_df = get_liquidation_call(TargetContract)

    reserves_status['variable_borrow_rate'] = reserves_status['variable_borrow_rate'].astype(int)
    reserves_status['stable_borrow_rate'] = reserves_status['stable_borrow_rate'].astype(int)
    reserves_status['liquidity_rate'] = reserves_status['liquidity_rate'].astype(int)
    liquidation_df['debt_to_cover'] = liquidation_df['debt_to_cover'].astype(int)
    liquidation_df['liquidated_collateral_amount'] = liquidation_df['liquidated_collateral_amount'].astype(int)
    interaction_df['amount'] = interaction_df['amount'].astype(int)

    interaction_df = interaction_df['action block_num index on_behalf_of reserve amount rate_mode rate'.split(' ')].copy()
    reserves_status = reserves_status[[
        'reserve', 'block_num', 'index',  
        'liquidity_rate', 'stable_borrow_rate', 'variable_borrow_rate', 
        'liquidity_index','variable_borrow_index'
    ]].copy()
    liquidation_df = liquidation_df[[
        'block_num', 'index', 'on_behalf_of', 
        'collateral_asset', 'debt_asset', 'debt_to_cover', 'liquidated_collateral_amount',
        'liquidator', 'receive_atoken']].copy()

    interaction_df['block_n_index'] = interaction_df.apply(combine_block_n_index, axis=1)
    reserves_status['block_n_index'] = reserves_status.apply(combine_block_n_index, axis=1)
    liquidation_df['block_n_index'] = liquidation_df.apply(combine_block_n_index, axis=1)

    interaction_df = interaction_df.sort_values('block_n_index').reset_index(drop=True)
    reserves_status = reserves_status.sort_values('block_n_index').reset_index(drop=True)


    # just give a random reserve address, will be swich in the following part
    for index in interaction_df.index:
        if interaction_df.loc[index, 'action'] == "LiquidationCall":
            interaction_df.loc[index, 'reserve'] = '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'

    # merge
    user_df = interaction_df.merge(reserves_status, on=['reserve'], how='left')
    change_token_address_to_name = lambda x: revert_token_dict[x] if x in revert_token_dict else x
    interaction_df['reserve'] = interaction_df['reserve'].apply(change_token_address_to_name).reset_index(drop=True)
    user_df['reserve'] = user_df['reserve'].apply(change_token_address_to_name).reset_index(drop=True)
    reserves_status['reserve'] = reserves_status['reserve'].apply(change_token_address_to_name).reset_index(drop=True)

    liquidation_df['collateral_asset'] = liquidation_df['collateral_asset'].apply(change_token_address_to_name)
    liquidation_df['debt_asset'] = liquidation_df['debt_asset'].apply(change_token_address_to_name)


    def get_liquidation_data(df_row):
        df_row = df_row.copy()
        if df_row['action'] != 'LiquidationCall': return df_row
        # collateral
        block_n_index_x = df_row['block_n_index_x']
        liquidation_row = liquidation_df[liquidation_df['block_n_index'] == block_n_index_x]
        collateral_asset = liquidation_row['collateral_asset'].values[0]
        tmp_reserves_status = reserves_status[\
            (reserves_status['reserve'] == collateral_asset) &\
            (reserves_status['block_n_index'] <= block_n_index_x)].copy().sort_values('block_n_index')
        tmp_reserves_status = tmp_reserves_status.iloc[-1, :]

        df_row['block_num_y'] = tmp_reserves_status['block_num']
        df_row['index_y'] = tmp_reserves_status['index']
        df_row['liquidity_rate'] = tmp_reserves_status['liquidity_rate']
        df_row['liquidity_index'] = tmp_reserves_status['liquidity_index']
        df_row['block_n_index_y'] = tmp_reserves_status['block_n_index']

        debt_asset = liquidation_row['debt_asset'].values[0]
        tmp_reserves_status = reserves_status[\
            (reserves_status['reserve'] == debt_asset) &\
            (reserves_status['block_n_index'] <= block_n_index_x)].copy().sort_values('block_n_index')
        tmp_reserves_status = tmp_reserves_status.iloc[-1, :]

        df_row['stable_borrow_rate'] = tmp_reserves_status['stable_borrow_rate']
        df_row['variable_borrow_rate'] = tmp_reserves_status['variable_borrow_rate']
        df_row['variable_borrow_index'] = tmp_reserves_status['variable_borrow_index']
        
        return df_row
    
    from_df = user_df[user_df['block_n_index_y'] <= user_df['block_n_index_x']]
    from_df = from_df.loc[from_df.groupby('block_n_index_x').block_n_index_y.idxmax()].reset_index(drop=True)
    from_df = from_df.apply(get_liquidation_data, axis=1)


    # a token
    collateral_dict = defaultdict(float)
    collatearl_able_dict = defaultdict(lambda :True)
    variable_debt_dict = defaultdict(float)
    stable_debt_dict = defaultdict(lambda : [None, None, None]) # amount, interest, start time

    sub_interaction_df = interaction_df[interaction_df['block_n_index'] <= until_block_n_index].copy()

    for index_i in sub_interaction_df.index:

        action_i = sub_interaction_df.loc[index_i, 'action']
        block_n_index = sub_interaction_df.loc[index_i, 'block_n_index']
        block_num = sub_interaction_df.loc[index_i, 'block_num']
        index = sub_interaction_df.loc[index_i, 'index']

        # block_time = get_block_time(block_num)
        before_data = from_df[from_df['block_n_index_x'] == block_n_index]#['amount'].values[0]
        liquidity_index = before_data['liquidity_index'].values[0]
        variable_borrow_index = before_data['variable_borrow_index'].values[0]
        stable_borrow_rate = before_data['stable_borrow_rate'].values[0]
        
        if action_i == "LiquidationCall":
            'collateral_asset', 'debt_asset', 'debt_to_cover', 'liquidated_collateral_amount',
            liquidation_i = liquidation_df[liquidation_df['block_n_index'] == block_n_index].copy().reset_index(drop=True)
            collateral_asset = liquidation_i.loc[0, 'collateral_asset']
            debt_asset = liquidation_i.loc[0, 'debt_asset']
            debt_to_cover = liquidation_i.loc[0, 'debt_to_cover']
            liquidated_collateral_amount = liquidation_i.loc[0, 'liquidated_collateral_amount']

            # a_token_amount_i = ray_div(liquidated_collateral_amount, liquidity_index)
            # collateral_dict[collateral_asset] -= a_token_amount_i

            collateral_in_original_unit, var_debt_in_original_unit, sta_debt_in_original_unit = get_token_value(
                block_num, index,
                collateral_dict, collatearl_able_dict, variable_debt_dict, stable_debt_dict,reserves_status
            )

            if var_debt_in_original_unit[debt_asset] < debt_to_cover:
                var_debt_to_liquidate = var_debt_in_original_unit[debt_asset]
                sta_debt_to_repay = debt_to_cover - var_debt_to_liquidate
            else:
                var_debt_to_liquidate = debt_to_cover
                sta_debt_to_repay = 0

            success, remaining_token = update_target_debt_data(
                    "Repay", block_num, var_debt_to_liquidate, debt_asset, 
                    "2", liquidity_index, variable_borrow_index, stable_borrow_rate,
                    collateral_dict, collatearl_able_dict, variable_debt_dict, stable_debt_dict
                )
            assert success

            if sta_debt_to_repay > 0:
                success, remaining_token = update_target_debt_data(
                    "Repay", block_num, sta_debt_to_repay, debt_asset, 
                    "1", liquidity_index, variable_borrow_index, stable_borrow_rate,
                    collateral_dict, collatearl_able_dict, variable_debt_dict, stable_debt_dict
                )
                assert success
            
            success, remaining_token = update_target_debt_data(
                "Withdraw", block_num, liquidated_collateral_amount, collateral_asset, 
                "-1", liquidity_index, variable_borrow_index, stable_borrow_rate,
                collateral_dict, collatearl_able_dict, variable_debt_dict, stable_debt_dict
            )
            assert success

        else:
            amount_i = sub_interaction_df.loc[index_i, 'amount']
            token_name_i = sub_interaction_df.loc[index_i, 'reserve']
            rate_mode_i = sub_interaction_df.loc[index_i, 'rate_mode']

            update_target_debt_data(action_i, block_num, amount_i, token_name_i, 
            rate_mode_i, liquidity_index, variable_borrow_index, stable_borrow_rate,
            collateral_dict, collatearl_able_dict, variable_debt_dict, stable_debt_dict)
    
    
    token_value_dicts = get_token_value(until_block_num, until_index, 
                            collateral_dict, collatearl_able_dict, variable_debt_dict, stable_debt_dict,
                            reserves_status,
                            fix_decimal=True
                        )

    # collateral, var_debt, sta_debt
    token_value_dicts = {i:j for i,j in zip(['collateral', 'var_debt', 'sta_debt'], token_value_dicts)}

    price_data = get_price_data(until_block_num, previous_block= 6424*1 + PreviousBlockForTrain)
    # price_data = get_price_data(until_block_num, previous_block=PreviousBlockForTrain)
    price_data['token0'] = price_data['token0'].apply(lambda x: 'weth' if x == 'eth' else x)
    price_data['token1'] = price_data['token1'].apply(lambda x: 'weth' if x == 'eth' else x)
    price_data = price_data[['block_num', 'oracle_name', 'token0', 'token1', 'current']]

    block_num_df = pd.DataFrame(
        range(
            price_data.block_num.min(), 
            until_block_num + 1
        ),
        columns=['block_num']
    )
    block_num_df.set_index('block_num', inplace=True)

    uniswapv3_price_dict = {}
    for token in ['usdt', 'dai', 'usdc']:
        sub_price_df = price_data[(price_data['oracle_name'] == 'uniswapv3') & (price_data['token1'] == token)].copy()
        sub_price_df[f'{token}'] = 1/sub_price_df['current']
        sub_price_df.set_index('block_num', inplace=True)
        sub_price_df = sub_price_df.merge(block_num_df, how='right', left_index=True, right_index=True)
        sub_price_df.fillna(method='ffill', inplace=True)
        sub_price_df.fillna(method='bfill', inplace=True)
        sub_price_df = sub_price_df[sub_price_df.index > (until_block_num-PreviousBlockForTrain-1)]
        uniswapv3_price_dict[token] = sub_price_df[token]
    chainlink_price_dict ={}
    for token in ['usdt', 'dai', 'usdc']:
        sub_price_df = price_data[(price_data['oracle_name'] == 'chainlink') & (price_data['token0'] == token)].copy()
        sub_price_df[f'{token}'] = sub_price_df['current']
        sub_price_df.set_index('block_num', inplace=True)
        sub_price_df = sub_price_df.merge(block_num_df, how='right', left_index=True, right_index=True)
        sub_price_df.fillna(method='ffill', inplace=True)
        sub_price_df.fillna(method='bfill', inplace=True)
        sub_price_df = sub_price_df[sub_price_df.index > (until_block_num-PreviousBlockForTrain-1)]
        chainlink_price_dict[token] = sub_price_df[token]

    collatearl_in_eth = 0
    debt_in_eth = 0
    for token_name, token_amount in token_value_dicts['collateral'].items():
        if token_name == 'weth':
            collatearl_in_eth += token_amount
        else:
            collatearl_in_eth += token_amount * chainlink_price_dict[token_name].loc[until_block_num]

    for token_name, token_amount in token_value_dicts['var_debt'].items():
        if token_name == 'weth':
            debt_in_eth += token_amount
        else:
            debt_in_eth += token_amount * chainlink_price_dict[token_name].loc[until_block_num]

    for token_name, token_amount in token_value_dicts['sta_debt'].items():
        if token_name == 'weth':
            debt_in_eth += token_amount
        else:
            debt_in_eth += token_amount * chainlink_price_dict[token_name].loc[until_block_num]

    collatearl_m_threshold_in_eth = 0
    debt_m_threshold_in_eth = 0
    for token_name, token_amount in token_value_dicts['collateral'].items():
        if token_name == 'weth':
            collatearl_m_threshold_in_eth += token_amount * liquidation_threshold_dict[token_name]
        else:
            collatearl_m_threshold_in_eth += token_amount * chainlink_price_dict[token_name].loc[until_block_num]  * liquidation_threshold_dict[token_name]

    for token_name, token_amount in token_value_dicts['var_debt'].items():
        if token_name == 'weth':
            debt_m_threshold_in_eth += token_amount
        else:
            debt_m_threshold_in_eth += token_amount * chainlink_price_dict[token_name].loc[until_block_num]

    for token_name, token_amount in token_value_dicts['sta_debt'].items():
        if token_name == 'weth':
            debt_m_threshold_in_eth += token_amount
        else:
            debt_m_threshold_in_eth += token_amount * chainlink_price_dict[token_name].loc[until_block_num]
    collatearl_m_threshold_in_eth, debt_m_threshold_in_eth

    testtt = interaction_df[['action', 'block_num', 'reserve', 'amount']]

    current_healthfactor = (collatearl_m_threshold_in_eth/debt_m_threshold_in_eth)

    used_token_list = []
    price_data_list = []
    price_name_list = []
    for asset_from in ['collateral', 'var_debt', 'sta_debt']:
        for token_name in ['usdc', 'usdt', 'dai']:
        # for token_name, token_amount in token_value_dicts[asset_from].items():
            if token_name == 'weth': continue
            if token_name not in used_token_list:
                used_token_list.append(token_name)
                price_data_list.append(chainlink_price_dict[token_name])
                price_name_list.append(f'chainlink_{token_name}')
                price_data_list.append(uniswapv3_price_dict[token_name])
                price_name_list.append(f'uniswapv3_{token_name}')
    var_train_df = pd.concat(price_data_list, axis=1)
    var_train_df.columns = price_name_list
    var_train_df = var_train_df.reset_index(drop=False)

    hf_actual_series = cal_hf(var_train_df, token_value_dicts, liquidation_threshold_dict)
    hf_actual_series.index = var_train_df['block_num']


    train_data = var_train_df.set_index('block_num')
    tmp_df = train_data.copy()
    log_f_diff = tmp_df.diff().dropna() 
    log_f_diff = log_f_diff.reset_index(drop=True)
    train_success = False

    trained_var = get_var_result(log_f_diff, maxlags=None)

    def mc_simulate(df, step=240):
        price_diff_prediction = pd.DataFrame(trained_var.simulate_var(step), columns=log_f_diff.columns)
        price_prediction = invert_transformation(tmp_df, price_diff_prediction) 
        hf_df = cal_hf(price_prediction, token_value_dicts, liquidation_threshold_dict)
        return hf_df

    mc_hf = pd.DataFrame(range(MCAmount)).parallel_apply(mc_simulate, args=(StepAhead,), axis=1).T
    horizontal_liquidation_pct = mc_hf.parallel_apply(cal_pct_be_liquidated, axis=1)#.plot()


    return hf_actual_series, horizontal_liquidation_pct

In [28]:
GoInt64 = ctypes.c_int64
GoInt = GoInt64

# pandarallel.initialize(progress_bar=False)

__library = cdll.LoadLibrary('../eth_crawler/library.so')
get_aave_log = __library.get_aave_log
get_aave_log.argtypes = [c_char_p, GoInt, GoInt]
get_aave_log.restype = c_char_p

res = get_aave_log(
    # data_source.encode(), 
    archive_node.encode(), 
    GoInt(13054560), 
    GoInt(13054560)
)
res

b'{"Topic0":"0x804c9b842b2748a22bb64b345453a3de7ca54a6ca45ce00d415894979e22897a","Topic1":"0x000000000000000000000000c02aaa39b223fe8d0a0e5c4f27ead9083c756cc2","Data":"0000000000000000000000000000000000000000000035da4b5da6313bb330ee0000000000000000000000000000000000000000001ebd0350cb23712ea3614800000000000000000000000000000000000000000004bd033ed8e61373b5e76d000000000000000000000000000000000000000003414ed00c2e635d33a0c4fa0000000000000000000000000000000000000000034269fdcb8432bee2e530a7","BlockNumber":"13054560","Index":"534"};{"Topic0":"0x3115d1449a7b732c986cba18244e897a450f61e1bb8d589cd2e69e6c8924f9f7","Topic1":"0x000000000000000000000000c02aaa39b223fe8d0a0e5c4f27ead9083c756cc2","Topic2":"0x000000000000000000000000cc9a0b7c43dc2a5f023bb9b738e45b0ef6b06e04","Topic3":"0x000000000000000000000000cc9a0b7c43dc2a5f023bb9b738e45b0ef6b06e04","Data":"000000000000000000000000000000000000000000000000083019dfc17b0000","BlockNumber":"13054560","Index":"538"};'

In [25]:
def gen_real_hf(reserves_status, TargetContract, ReservesStatusEnd, StepAhead, ReservesStatusEndIndex, PreviousBlockForTrain):

    # reserves_status = get_reserves_status()
    reserves_status = reserves_status.copy()
    
    latest_block_num = reserves_status['block_num'].max()
    interaction_df = get_interaction_data(TargetContract)

    until_block_num = latest_block_num
    until_block_num = ReservesStatusEnd
    until_index = ReservesStatusEndIndex

    reserves_status = reserves_status[reserves_status['block_num'] <= ReservesStatusEnd]
    until_block_n_index = combine_block_n_index({'block_num': until_block_num, 'index': until_index})
    # liquidation_index = until_index

    # Start Getting Data #####################################
    liquidation_df = get_liquidation_call(TargetContract)

    # reserves_status['variable_borrow_rate'] = reserves_status['variable_borrow_rate'].astype(int)
    # reserves_status['stable_borrow_rate'] = reserves_status['stable_borrow_rate'].astype(int)
    # reserves_status['liquidity_rate'] = reserves_status['liquidity_rate'].astype(int)
    # liquidation_df['debt_to_cover'] = liquidation_df['debt_to_cover'].astype(int)
    # liquidation_df['liquidated_collateral_amount'] = liquidation_df['liquidated_collateral_amount'].astype(int)
    # interaction_df['amount'] = interaction_df['amount'].astype(int)

    interaction_df = interaction_df['action block_num index on_behalf_of reserve amount rate_mode rate'.split(' ')].copy()
    reserves_status = reserves_status[[
        'reserve', 'block_num', 'index',  
        'liquidity_rate', 'stable_borrow_rate', 'variable_borrow_rate', 
        'liquidity_index','variable_borrow_index'
    ]].copy()
    liquidation_df = liquidation_df[[
        'block_num', 'index', 'on_behalf_of', 
        'collateral_asset', 'debt_asset', 'debt_to_cover', 'liquidated_collateral_amount',
        'liquidator', 'receive_atoken']].copy()

    interaction_df['block_n_index'] = interaction_df.apply(combine_block_n_index, axis=1)
    reserves_status['block_n_index'] = reserves_status.apply(combine_block_n_index, axis=1)
    liquidation_df['block_n_index'] = liquidation_df.apply(combine_block_n_index, axis=1)

    interaction_df = interaction_df.sort_values('block_n_index').reset_index(drop=True)
    reserves_status = reserves_status.sort_values('block_n_index').reset_index(drop=True)


    # just give a random reserve address, will be swich in the following part
    for index in interaction_df.index:
        if interaction_df.loc[index, 'action'] == "LiquidationCall":
            interaction_df.loc[index, 'reserve'] = '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'

    # merge


    change_token_address_to_name = lambda x: revert_token_dict[x] if x in revert_token_dict else x
    interaction_df['reserve'] = interaction_df['reserve'].apply(change_token_address_to_name).reset_index(drop=True)
    reserves_status['reserve'] = reserves_status['reserve'].apply(change_token_address_to_name).reset_index(drop=True)
    liquidation_df['collateral_asset'] = liquidation_df['collateral_asset'].apply(change_token_address_to_name)
    liquidation_df['debt_asset'] = liquidation_df['debt_asset'].apply(change_token_address_to_name)


    collateral_dict = defaultdict(float)
    collatearl_able_dict = defaultdict(lambda :True)
    variable_debt_dict = defaultdict(float)
    stable_debt_dict = defaultdict(lambda : [None, None, None]) # amount, interest, start time

    print(interaction_df)

    sub_interaction_df = interaction_df[interaction_df['block_n_index'] <= until_block_n_index].copy()
    curent_block_index = 0
    hf_list = []
    block_list = []
    have_data = False
    block_n_index = None
    interaction_block_list = sub_interaction_df['block_num'].to_list()
    for tmp_block_num in tqdm.tqdm(reserves_status['block_num'].unique()):
        if tmp_block_num not in interaction_block_list and tmp_block_num < (ReservesStatusEnd - StepAhead):
            continue
        until_block_num = tmp_block_num
        tmp_reserves_status = reserves_status[reserves_status['block_num'] <= tmp_block_num]#.copy()
        tmp_index = 9999
        tmp_block_n_index = combine_block_n_index(dict(block_num=tmp_block_num, index=tmp_index))
        sub_interaction_df = interaction_df[(interaction_df['block_n_index'] > curent_block_index) & (interaction_df['block_n_index'] <= tmp_block_n_index)].copy()
        
        update_value = False
        for index_i in sub_interaction_df.index:
            update_value = True
            print(sub_interaction_df.loc[index_i, :])
            print(tmp_reserves_status.iloc[-2:,:])
            have_data = True
            action_i = sub_interaction_df.loc[index_i, 'action']
            block_n_index = sub_interaction_df.loc[index_i, 'block_n_index']
            block_num = sub_interaction_df.loc[index_i, 'block_num']
            index = sub_interaction_df.loc[index_i, 'index']

            # block_time = get_block_time(block_num)
            # before_data = from_df[from_df['block_n_index_x'] == block_n_index]#['amount'].values[0]
            # liquidity_index = tmp_reserves_status['liquidity_index'].values[-1]
            # variable_borrow_index = tmp_reserves_status['variable_borrow_index'].values[0]
            # stable_borrow_rate = tmp_reserves_status['stable_borrow_rate'].values[0]
            
            if action_i == "LiquidationCall":
                'collateral_asset', 'debt_asset', 'debt_to_cover', 'liquidated_collateral_amount',
                liquidation_i = liquidation_df[liquidation_df['block_n_index'] == block_n_index].copy().reset_index(drop=True)
                print(liquidation_i)
                collateral_asset = liquidation_i.loc[0, 'collateral_asset']
                debt_asset = liquidation_i.loc[0, 'debt_asset']
                debt_to_cover = liquidation_i.loc[0, 'debt_to_cover']
                liquidated_collateral_amount = liquidation_i.loc[0, 'liquidated_collateral_amount']

                liquidity_index = tmp_reserves_status[(tmp_reserves_status['reserve']==collateral_asset) & (tmp_reserves_status['block_n_index']<=block_n_index)]['liquidity_index'].values[-1]
                variable_borrow_index = tmp_reserves_status[(tmp_reserves_status['reserve']==debt_asset) & (tmp_reserves_status['block_n_index']<=block_n_index)]['variable_borrow_index'].values[-1]
                stable_borrow_rate = tmp_reserves_status[(tmp_reserves_status['reserve']==debt_asset) & (tmp_reserves_status['block_n_index']<=block_n_index)]['stable_borrow_rate'].values[-1]

                # a_token_amount_i = ray_div(liquidated_collateral_amount, liquidity_index)
                # collateral_dict[collateral_asset] -= a_token_amount_i

                collateral_in_original_unit, var_debt_in_original_unit, sta_debt_in_original_unit = get_token_value(
                    block_num, index,
                    collateral_dict, collatearl_able_dict, variable_debt_dict, stable_debt_dict,reserves_status
                )

                if var_debt_in_original_unit[debt_asset] < debt_to_cover:
                    var_debt_to_liquidate = var_debt_in_original_unit[debt_asset]
                    sta_debt_to_repay = debt_to_cover - var_debt_to_liquidate
                else:
                    var_debt_to_liquidate = debt_to_cover
                    sta_debt_to_repay = 0

                success, remaining_token = update_target_debt_data(
                        "Repay", block_num, var_debt_to_liquidate, debt_asset, 
                        "2", liquidity_index, variable_borrow_index, stable_borrow_rate,
                        collateral_dict, collatearl_able_dict, variable_debt_dict, stable_debt_dict
                    )
                assert success

                if sta_debt_to_repay > 0:
                    success, remaining_token = update_target_debt_data(
                        "Repay", block_num, sta_debt_to_repay, debt_asset, 
                        "1", liquidity_index, variable_borrow_index, stable_borrow_rate,
                        collateral_dict, collatearl_able_dict, variable_debt_dict, stable_debt_dict
                    )
                    assert success
                
                success, remaining_token = update_target_debt_data(
                    "Withdraw", block_num, liquidated_collateral_amount, collateral_asset, 
                    "-1", liquidity_index, variable_borrow_index, stable_borrow_rate,
                    collateral_dict, collatearl_able_dict, variable_debt_dict, stable_debt_dict
                )
                assert success

            else:
                amount_i = sub_interaction_df.loc[index_i, 'amount']
                token_name_i = sub_interaction_df.loc[index_i, 'reserve']
                rate_mode_i = sub_interaction_df.loc[index_i, 'rate_mode']

                liquidity_index = tmp_reserves_status[(tmp_reserves_status['reserve']==token_name_i) & (tmp_reserves_status['block_n_index']<=block_n_index)]['liquidity_index'].values[-1]
                variable_borrow_index = tmp_reserves_status[(tmp_reserves_status['reserve']==token_name_i) & (tmp_reserves_status['block_n_index']<=block_n_index)]['variable_borrow_index'].values[-1]
                stable_borrow_rate = tmp_reserves_status[(tmp_reserves_status['reserve']==token_name_i) & (tmp_reserves_status['block_n_index']<=block_n_index)]['stable_borrow_rate'].values[-1]

                update_target_debt_data(action_i, block_num, amount_i, token_name_i, 
                rate_mode_i, liquidity_index, variable_borrow_index, stable_borrow_rate,
                collateral_dict, collatearl_able_dict, variable_debt_dict, stable_debt_dict)
            # print(sub_interaction_df.loc[index_i, :])
            
            print("liquidity_index, variable_borrow_index, stable_borrow_rate")
            print(liquidity_index, variable_borrow_index, stable_borrow_rate)
            print("collateral_dict, collatearl_able_dict, variable_debt_dict, stable_debt_dict")
            print(collateral_dict, collatearl_able_dict, variable_debt_dict, stable_debt_dict)
            
            print('---------------------------------------------------------------')
        
        if update_value:
            print(get_token_value(until_block_num, until_index, 
                                collateral_dict, collatearl_able_dict, variable_debt_dict, stable_debt_dict,
                                reserves_status,
                                fix_decimal=True
                            ))

        if block_n_index is None:
            continue
        curent_block_index = block_n_index
        
        if not have_data: continue
        if len(variable_debt_dict) == 0 and len(stable_debt_dict) == 0: continue
        # if : continue
        if tmp_block_num < (ReservesStatusEnd - StepAhead): continue
        if tmp_block_num > ReservesStatusEnd: break

        
        token_value_dicts = get_token_value(until_block_num, until_index, 
                                collateral_dict, collatearl_able_dict, variable_debt_dict, stable_debt_dict,
                                reserves_status,
                                fix_decimal=True
                            )

        # collateral, var_debt, sta_debt
        token_value_dicts = {i:j for i,j in zip(['collateral', 'var_debt', 'sta_debt'], token_value_dicts)}
        print(token_value_dicts)

        price_data = get_price_data(until_block_num, previous_block= 6424*1)
        # price_data = get_price_data(until_block_num, previous_block=PreviousBlockForTrain)
        price_data['token0'] = price_data['token0'].apply(lambda x: 'weth' if x == 'eth' else x)
        price_data['token1'] = price_data['token1'].apply(lambda x: 'weth' if x == 'eth' else x)
        price_data = price_data[['block_num', 'oracle_name', 'token0', 'token1', 'current']]

        block_num_df = pd.DataFrame(
            range(
                price_data.block_num.min(), 
                until_block_num + 1
            ),
            columns=['block_num']
        )
        block_num_df.set_index('block_num', inplace=True)

        uniswapv3_price_dict = {}
        for token in ['usdt', 'dai', 'usdc']:
            sub_price_df = price_data[(price_data['oracle_name'] == 'uniswapv3') & (price_data['token1'] == token)].copy()
            sub_price_df[f'{token}'] = 1/sub_price_df['current']
            sub_price_df.set_index('block_num', inplace=True)
            sub_price_df = sub_price_df.merge(block_num_df, how='right', left_index=True, right_index=True)
            sub_price_df.fillna(method='ffill', inplace=True)
            sub_price_df.fillna(method='bfill', inplace=True)
            sub_price_df = sub_price_df[sub_price_df.index > (until_block_num-PreviousBlockForTrain-1)]
            uniswapv3_price_dict[token] = sub_price_df[token]
        chainlink_price_dict ={}
        for token in ['usdt', 'dai', 'usdc']:
            sub_price_df = price_data[(price_data['oracle_name'] == 'chainlink') & (price_data['token0'] == token)].copy()
            sub_price_df[f'{token}'] = sub_price_df['current']
            sub_price_df.set_index('block_num', inplace=True)
            sub_price_df = sub_price_df.merge(block_num_df, how='right', left_index=True, right_index=True)
            sub_price_df.fillna(method='ffill', inplace=True)
            sub_price_df.fillna(method='bfill', inplace=True)
            sub_price_df = sub_price_df[sub_price_df.index > (until_block_num-PreviousBlockForTrain-1)]
            chainlink_price_dict[token] = sub_price_df[token]

        collatearl_in_eth = 0
        debt_in_eth = 0
        for token_name, token_amount in token_value_dicts['collateral'].items():
            if token_name == 'weth':
                collatearl_in_eth += token_amount
            else:
                collatearl_in_eth += token_amount * chainlink_price_dict[token_name].loc[until_block_num]

        for token_name, token_amount in token_value_dicts['var_debt'].items():
            if token_name == 'weth':
                debt_in_eth += token_amount
            else:
                debt_in_eth += token_amount * chainlink_price_dict[token_name].loc[until_block_num]

        for token_name, token_amount in token_value_dicts['sta_debt'].items():
            if token_name == 'weth':
                debt_in_eth += token_amount
            else:
                debt_in_eth += token_amount * chainlink_price_dict[token_name].loc[until_block_num]

        collatearl_m_threshold_in_eth = 0
        debt_m_threshold_in_eth = 0
        for token_name, token_amount in token_value_dicts['collateral'].items():
            if token_name == 'weth':
                collatearl_m_threshold_in_eth += token_amount * liquidation_threshold_dict[token_name]
            else:
                collatearl_m_threshold_in_eth += token_amount * chainlink_price_dict[token_name].loc[until_block_num]  * liquidation_threshold_dict[token_name]

        for token_name, token_amount in token_value_dicts['var_debt'].items():
            if token_name == 'weth':
                debt_m_threshold_in_eth += token_amount
            else:
                debt_m_threshold_in_eth += token_amount * chainlink_price_dict[token_name].loc[until_block_num]

        for token_name, token_amount in token_value_dicts['sta_debt'].items():
            if token_name == 'weth':
                debt_m_threshold_in_eth += token_amount
            else:
                debt_m_threshold_in_eth += token_amount * chainlink_price_dict[token_name].loc[until_block_num]
        collatearl_m_threshold_in_eth, debt_m_threshold_in_eth

        testtt = interaction_df[['action', 'block_num', 'reserve', 'amount']]

        current_healthfactor = (collatearl_m_threshold_in_eth/debt_m_threshold_in_eth)

        used_token_list = []
        price_data_list = []
        price_name_list = []
        for asset_from in ['collateral', 'var_debt', 'sta_debt']:
            for token_name in ['usdc', 'usdt', 'dai']:
            # for token_name, token_amount in token_value_dicts[asset_from].items():
                if token_name == 'weth': continue
                if token_name not in used_token_list:
                    used_token_list.append(token_name)
                    price_data_list.append(chainlink_price_dict[token_name])
                    price_name_list.append(f'chainlink_{token_name}')
                    price_data_list.append(uniswapv3_price_dict[token_name])
                    price_name_list.append(f'uniswapv3_{token_name}')

        var_train_df = pd.concat(price_data_list, axis=1)
        var_train_df.columns = price_name_list
        var_train_df = var_train_df.reset_index(drop=False)

        hf_actual_series = cal_hf(var_train_df, token_value_dicts, liquidation_threshold_dict)
        block_list.append(tmp_block_num)
        hf_list.append(hf_actual_series.iloc[-1])
    return tmp_block_num, hf_list

In [ ]:
change_token_address_to_name = lambda x: revert_token_dict[x] if x in revert_token_dict else x
reserves_status = get_reserves_status()
liquidation_data = get_liquidation_call()
liquidation_data = liquidation_data.iloc[:,2:]

np.random.seed(10)
liquidation_data = liquidation_data.loc[np.random.choice(liquidation_data.index, 100)]
i = 0
price_data_start_from = 12469311
def vaidate_data(df_row):
    interaction_df = get_interaction_data(df_row.loc['on_behalf_of'])
    token_ok = interaction_df['reserve'].apply(change_token_address_to_name).apply(lambda x: len(x) <= 10).all()
    collateral_ok = interaction_df[interaction_df['action']=='Deposit']['reserve'].apply(change_token_address_to_name).apply(lambda x: x != 'usdt').all()
    no_swap = interaction_df['action'].apply(lambda x: x != 'Swap').all()
    have_price_data = interaction_df['block_num'].apply(lambda x: x > price_data_start_from).all()
    return token_ok and collateral_ok and no_swap and have_price_data
valid_data_bool = liquidation_data.parallel_apply(vaidate_data, axis=1)
liquidation_data_ok = liquidation_data[valid_data_bool]
liquidation_data_ok

In [ ]:
i = 3
step_ahead = 10
step_before = -1
mc_amount = 100
reserves_status_end_index = 9999
previous_block_for_train = 10


tmp_data = liquidation_data_ok.iloc[i,:]

hf_list = gen_real_hf(
    reserves_status[(reserves_status['block_num']>(13550124-6424)) & (reserves_status['block_num']<tmp_data['block_num'] - step_before)],
    TargetContract = tmp_data['on_behalf_of'],
    ReservesStatusEnd = tmp_data['block_num'] - step_before,
    StepAhead = step_ahead,
    ReservesStatusEndIndex = reserves_status_end_index,
    PreviousBlockForTrain = previous_block_for_train
)
tmp_block_num, hf_list = hf_list

In [27]:
liquidation_data_ok.iloc[i,:]

index                                                                  137
on_behalf_of                    0x5921b5992f4ee410c39829880aa3669b5e0f8923
collateral_asset                0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
debt_asset                      0xdac17f958d2ee523a2206206994597c13d831ec7
debt_to_cover                                                 2212865686.0
liquidated_collateral_amount                          593792297834835712.0
liquidator                      0x3909336de913344701c6f096502d26208210b39f
receive_atoken                                                        True
transaction_hash                                                          
block_num                                                         13797462
Name: 1270, dtype: object

In [26]:
i = 11
step_ahead = 20
step_before = -1
mc_amount = 100
reserves_status_end_index = 9999
previous_block_for_train = 10


tmp_data = liquidation_data_ok.iloc[i,:]

hf_list = gen_real_hf(
    reserves_status[reserves_status['block_num']<15366765],
    TargetContract = tmp_data['on_behalf_of'],
    ReservesStatusEnd = 15366765,
    StepAhead = step_ahead,
    ReservesStatusEndIndex = reserves_status_end_index,
    PreviousBlockForTrain = previous_block_for_train
)
tmp_block_num, hf_list = hf_list

             action  block_num  index  \
0           Deposit   13022008    602   
1            Borrow   13022789    330   
2           Deposit   13022946    191   
3            Borrow   13028209    303   
4           Deposit   13028235    493   
5            Borrow   13034986    293   
6           Deposit   13035107    290   
7             Repay   13054584    218   
8            Borrow   13081139    500   
9           Deposit   13081160    372   
10           Borrow   13088270    547   
11            Repay   13125959    220   
12           Borrow   13126835    328   
13           Borrow   13157553    159   
14          Deposit   13157631    553   
15           Borrow   13159409    246   
16            Repay   13196651    122   
17           Borrow   13223444    550   
18           Borrow   13236503    407   
19  LiquidationCall   13273785     66   
20           Borrow   13378611    190   
21          Deposit   13403198    143   
22           Borrow   13403205    197   
23          Depo

 53%|█████████████████████████████████████████████████████▊                                                | 340636/646321 [00:01<00:01, 273844.99it/s]

action                                              Deposit
block_num                                          13022008
index                                                   602
on_behalf_of     0x5921b5992f4ee410c39829880aa3669b5e0f8923
reserve                                                weth
amount                                2630000000000000000.0
rate_mode                                                -1
rate                                                   -1.0
block_n_index                                13022008000602
Name: 0, dtype: object
                                           reserve  block_num  index  \
497901  0x514910771af9ca656af840dff83e8264ecf986ca   13021995    502   
497902                                        weth   13022008    596   

        liquidity_rate  stable_borrow_rate  variable_borrow_rate  \
497901    5.752289e+22        3.258295e+25          1.808067e+24   
497902    2.796632e+23        3.748160e+25          5.985284e+24   

        liquidi

 57%|██████████████████████████████████████████████████████████▋                                            | 368021/646321 [00:06<00:17, 15922.27it/s]

(defaultdict(<class 'float'>, {'weth': 5.300137387869974}), defaultdict(<class 'float'>, {'usdc': 10376.47918221116, 'usdt': 2804.1847356519143}), defaultdict(<class 'float'>, {}))
action                                               Borrow
block_num                                          13223444
index                                                   550
on_behalf_of     0x5921b5992f4ee410c39829880aa3669b5e0f8923
reserve                                                usdc
amount                                          400000000.0
rate_mode                                                 2
rate                           39424462724765327338504192.0
block_n_index                                13223444000550
Name: 17, dtype: object
                                           reserve  block_num  index  \
537884  0x514910771af9ca656af840dff83e8264ecf986ca   13223425     84   
537885                                        usdc   13223444    548   

        liquidity_rate  stable_borrow_

...

 93%|███████████████████████████████████████████████████████████████████████████████████████████████▊       | 601494/646321 [00:18<00:01, 36333.58it/s]

(defaultdict(<class 'float'>, {'weth': 2.9520347540112124}), defaultdict(<class 'float'>, {'usdc': 8536.706652136145, 'usdt': 292.71472402265783, 'weth': 0.20094682904044234}), defaultdict(<class 'float'>, {}))
action                                      LiquidationCall
block_num                                          15197121
index                                                    66
on_behalf_of     0x5921b5992f4ee410c39829880aa3669b5e0f8923
reserve                                                usdc
amount                                                 -1.0
rate_mode                                                -1
rate                                                   -1.0
block_n_index                                15197121000066
Name: 34, dtype: object
       reserve  block_num  index  liquidity_rate  stable_borrow_rate  \
896299    weth   15197121     60    8.964845e+24        5.748479e+25   
896300    weth   15197121     61    8.964849e+24        5.748480e+25   

        

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████▋| 644546/646321 [00:19<00:00, 40867.42it/s]

(defaultdict(<class 'float'>, {'weth': 2.8682082215353795}), defaultdict(<class 'float'>, {'usdc': 8560.918275543545, 'usdt': 293.9387821683317, 'weth': 0.11855776644404142}), defaultdict(<class 'float'>, {}))
{'collateral': defaultdict(<class 'float'>, {'weth': 2.871000345697484}), 'var_debt': defaultdict(<class 'float'>, {'usdc': 8571.046015821226, 'usdt': 294.4676785217717, 'weth': 0.11875011016056758}), 'sta_debt': defaultdict(<class 'float'>, {})}
{'collateral': defaultdict(<class 'float'>, {'weth': 2.871000345697484}), 'var_debt': defaultdict(<class 'float'>, {'usdc': 8571.046015821226, 'usdt': 294.4676785217717, 'weth': 0.11875011016056758}), 'sta_debt': defaultdict(<class 'float'>, {})}
{'collateral': defaultdict(<class 'float'>, {'weth': 2.871000345697484}), 'var_debt': defaultdict(<class 'float'>, {'usdc': 8571.047531060756, 'usdt': 294.4676785217717, 'weth': 0.11875011016056758}), 'sta_debt': defaultdict(<class 'float'>, {})}


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 646321/646321 [00:21<00:00, 30123.61it/s]


In [ ]:
# hf_list = [i for i in hf_list if i < 100]
pd.DataFrame(hf_list)

In [ ]:
pd.DataFrame(hf_list).plot()

In [ ]:
(reserves_status['block_num']>13550124) & (reserves_status['block_num']<tmp_data['block_num'] - step_before)

In [ ]:
change_token_address_to_name = lambda x: revert_token_dict[x] if x in revert_token_dict else x

In [ ]:
liquidation_data = get_liquidation_call()
liquidation_data = liquidation_data.iloc[:,2:]

np.random.seed(10)
liquidation_data = liquidation_data.loc[np.random.choice(liquidation_data.index, 100)]
liquidation_data

In [ ]:
i = 0
price_data_start_from = 12469311
def vaidate_data(df_row):
    interaction_df = get_interaction_data(df_row.loc['on_behalf_of'])
    token_ok = interaction_df['reserve'].apply(change_token_address_to_name).apply(lambda x: len(x) <= 10).all()
    collateral_ok = interaction_df[interaction_df['action']=='Deposit']['reserve'].apply(change_token_address_to_name).apply(lambda x: x != 'usdt').all()
    no_swap = interaction_df['action'].apply(lambda x: x != 'Swap').all()
    have_price_data = interaction_df['block_num'].apply(lambda x: x > price_data_start_from).all()
    return token_ok and collateral_ok and no_swap and have_price_data
valid_data_bool = liquidation_data.parallel_apply(vaidate_data, axis=1)

In [ ]:
liquidation_data_ok = liquidation_data[valid_data_bool]
liquidation_data_ok

In [ ]:
tmp_data['block_num']

In [ ]:
i = 3
step_ahead = 500
step_before = -1
mc_amount = 100
reserves_status_end_index = 9999
previous_block_for_train = 6424

tmp_data = liquidation_data_ok.iloc[i,:]

hf_actual_series, horizontal_liquidation_pct = gen_hf_chart(
    TargetContract = tmp_data['on_behalf_of'],
    ReservesStatusEnd = tmp_data['block_num'] - step_before,
    StepAhead = step_ahead,
    MCAmount = mc_amount,
    ReservesStatusEndIndex = reserves_status_end_index,
    PreviousBlockForTrain = previous_block_for_train
)

In [ ]:
interaction_df = get_interaction_data(tmp_data['on_behalf_of'])
interaction_df['reserve'] = interaction_df['reserve'].apply(change_token_address_to_name)
interaction_df.sort_values('block_num')[['block_num', 'action', 'reserve']]

In [ ]:
hf_actual_series.reset_index(drop=True).plot()

In [ ]:
hf_actual_series

In [ ]:
horizontal_liquidation_pct.plot()